In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser, JsonKeyOutputFunctionsParser
from langchain_community.document_loaders.web_base import WebBaseLoader

texto = """
A Microsoft foi fundada em 4 de abril de 1975 por Bill Gates e Paul Allen, em Albuquerque, 
no estado do Novo México. O nome "Microsoft" é uma combinação das palavras "microcomputador" 
e "software", refletindo o foco da empresa em software para computadores pessoais.
O primeiro grande projeto da Microsoft foi a criação de um sistema operacional para o 
computador Altair 8800, um dos primeiros microcomputadores disponíveis comercialmente. 
O sistema, denominado Altair BASIC, foi desenvolvido em parceria com a MITS 
(Micro Instrumentation and Telemetry Systems) e foi um marco inicial para a Microsoft.
Em 1980, a empresa firmou um contrato significativo com a IBM para fornecer o sistema 
operacional para o novo PC da IBM, o que levou à criação do MS-DOS. Esse contrato foi um 
ponto de virada para a Microsoft, impulsionando sua expansão e dominando o mercado de 
sistemas operacionais para PCs nos anos seguintes.
Com o sucesso do MS-DOS, a Microsoft se consolidou como líder no setor de software e, 
em 1985, lançou o Windows, um sistema operacional gráfico que viria a se tornar a base 
de sua supremacia no mercado de sistemas operacionais para desktop.
"""


class Event(BaseModel):
    """Modelo de evento para armazenar informações sobre eventos históricos."""
    date: str = Field(description="Data do evento no formato AAAA-MM-DD")
    event: str = Field(description="Descrição do evento extraído do texto")

class EventsList(BaseModel):
    """Modelo para armazenar uma lista de eventos históricos."""
    events: List[Event] = Field(description="Lista de eventos históricos extraídos do texto")

tool_event = convert_to_openai_function(EventsList)

model = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um assistente que extrai eventos históricos de um texto."),
        ("user", "Por favor, extraia os eventos históricos do seguinte texto: {texto}"),
    ]
)

chain = prompt | model.bind(functions=[tool_event], function_call="auto") | JsonOutputFunctionsParser()
# print(chain.invoke({"texto": texto}))



# Extrair dados da web
loader = WebBaseLoader("https://www.techtudo.com.br")
page = loader.load()


class WebEvent(BaseModel):
    """Modelo de evento para armazenar informações sobre eventos históricos extraídos da web."""
    title: str = Field(description="Título do evento")
    author: str = Field(description="Autor do evento")
    url: str = Field(description="URL do evento")
    date: str = Field(description="Data do evento no formato AAAA-MM-DD")
    description: str = Field(description="Descrição do evento extraído da página web")

class WebEventsList(BaseModel):
    """Modelo para armazenar uma lista de postagens históricos extraídos da web."""
    posts: List[WebEvent] = Field(description="Lista de postagens extraídos da página web")


tool_blog_event = convert_to_openai_function(WebEventsList)
web_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um assistente que extrai postagens de blogs de tecnologia."),
        ("user", "Por favor, extraia as postagens do blog da seguinte página: {page}"),
    ]
)

web_chain = web_prompt | model.bind(functions=[tool_blog_event], function_call="auto") | JsonOutputFunctionsParser()
# print(web_chain.invoke({"page": page}))

web_chain_2 = web_prompt | model.bind(functions=[tool_blog_event], function_call="auto") | JsonKeyOutputFunctionsParser(key_name="posts")
print(web_chain_2.invoke({"page": page}))

[{'title': 'Google Gemini: 7 coisas que você não sabia sobre a IA do Google', 'author': 'TechTudo', 'url': 'https://www.techtudo.com.br/google-gemini-7-coisas-que-voce-nao-sabia-sobre-ia-do-google', 'date': '2025-08-15', 'description': 'Gemini é a IA multimodal do Google que substituiu o Bard, oferecendo recursos da criação de textos e imagens à linguagem de programação; veja algumas curiosidades'}, {'title': 'Flamengo x Bayern ao vivo: como assistir ao jogo do Mundial de Clubes', 'author': 'TechTudo', 'url': 'https://www.techtudo.com.br/flamengo-bayern-ao-vivo-como-assistir-jogo-mundial-clubes', 'date': '2025-08-15', 'description': ''}, {'title': 'Como vai ser o GTA 6? Tudo que se sabe até agora e o que esperar do game', 'author': 'TechTudo', 'url': 'https://www.techtudo.com.br/como-vai-ser-gta-6-tudo-que-se-sabe-ate-agora-e-o-que-esperar-do-game', 'date': '2025-08-15', 'description': ''}, {'title': 'Como bloquear iPhone pelo iCloud? Confira o passo a passo', 'author': 'TechTudo', 'ur